# SARIMAX diario — Grid Search con Yeo-Johnson + selección de exógenas por VIF (pre-CV) + Rolling-Origin CV (5 folds)

**Objetivo:** dejar un notebook *limpio* para:
1) aplicar **Yeo-Johnson** al target (con **inversa** para evaluar en escala original),
2) seleccionar variables exógenas **solo con el set de train (antes del CV)** usando **poda por VIF**,
3) correr **Grid Search** de SARIMAX con **Rolling-Origin CV** de **5 pliegues** (horizonte 7 días),
4) calcular **MAPE/RMSE** promedio en CV,
5) ejecutar **Ljung-Box** con **lag=10** sobre los **residuos concatenados** de las predicciones de CV.

> Nota:  
> - La **selección de variables** se hace **una sola vez** con `df_train`.  

In [2]:
# =========================
# Imports / Setup
# =========================
import warnings
warnings.filterwarnings("ignore")
import re
import numpy as np
import pandas as pd
from pathlib import Path
from itertools import product
from tqdm.auto import tqdm

from scipy import stats
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.stats.diagnostic import acorr_ljungbox
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
from IPython.display import display

# Proyecto
from afg_puc import config

pd.set_option("display.max_columns", None)
pd.set_option("display.float_format", lambda x: f"{x:,.6f}")

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)


In [3]:
df_consol = pd.read_parquet(config.PROCESSED_DATA_DIR / "df_consol.parquet", engine="pyarrow")
df_consol.index = pd.to_datetime(df_consol.index)
df_consol = df_consol.sort_index()

df_train= df_consol.iloc[:-63]

dow = df_train.index.weekday
df_train["cercania_lunes"] = (0 - dow) % 7
df_train["lejanania_lunes_pas"] = (dow - 0) % 7

TARGET_COLS = ["oro", "plata", "cobre", "petroleo_brent", "gas_natural"]

print("TRAIN diario (excluye últimos 63 dias):", len(df_train), "|", df_train.index.min(), "->", df_train.index.max())
print("HOLDOUT 63 dias:", len(df_consol.loc[df_consol.index > df_train.index.max()]), "|", df_consol.loc[df_consol.index > df_train.index.max()].index.min(), "->", df_consol.loc[df_consol.index > df_train.index.max()].index.max())

TRAIN diario (excluye últimos 63 dias): 4700 | 2007-07-30 00:00:00 -> 2025-08-01 00:00:00
HOLDOUT 63 dias: 63 | 2025-08-04 00:00:00 -> 2025-10-29 00:00:00


## Variables exógenas base (por asset)

In [4]:
VARS_CSV_PATH = Path(config.PROCESSED_DATA_DIR) / "sarimax_model_vars_diario.csv"
print("CSV (OUTPUT) vars:", VARS_CSV_PATH)

# target por asset (ajusta si tus targets tienen otro nombre)
TARGET_COL_MAP = {
    "oro": "oro",
    "plata": "plata",
    "cobre": "cobre",
    "gas_natural": "gas_natural",
    "petroleo_brent": "petroleo_brent",
}

# columnas numéricas disponibles en train
NUM_COLS = df_train.select_dtypes(include=[np.number]).columns.tolist()

# Mapa asset -> lista candidatos (todas las numéricas menos su target)
EXOG_CANDIDATES_MAP = {}
for asset, target_col in TARGET_COL_MAP.items():
    EXOG_CANDIDATES_MAP[asset] = [c for c in NUM_COLS if c != target_col]

print("assets:", sorted(EXOG_CANDIDATES_MAP.keys()))
for a in sorted(EXOG_CANDIDATES_MAP.keys()):
    print(f"{a}: {len(EXOG_CANDIDATES_MAP[a])} candidatos (numéricas sin target)")

CSV (OUTPUT) vars: C:\Users\crsar\OneDrive - Universidad Católica de Chile\model\Proyect - Magister\code\afg_puc\data\processed\sarimax_model_vars_diario.csv
assets: ['cobre', 'gas_natural', 'oro', 'petroleo_brent', 'plata']
cobre: 70 candidatos (numéricas sin target)
gas_natural: 70 candidatos (numéricas sin target)
oro: 70 candidatos (numéricas sin target)
petroleo_brent: 70 candidatos (numéricas sin target)
plata: 70 candidatos (numéricas sin target)


## Helpers: limpieza numérica, Yeo-Johnson (transform + inversa), VIF

In [5]:
def _to_numeric_df(df: pd.DataFrame) -> pd.DataFrame:
    """Deja solo columnas numéricas, limpia inf y completa NA sin usar 'bfill' (para evitar mirar hacia adelante)."""
    X = (
        df.select_dtypes(include=[np.number])
          .replace([np.inf, -np.inf], np.nan)
          .ffill()
    )
    # completar NA remanentes con mediana (calculada en el mismo dataframe)
    med = X.median(numeric_only=True)
    X = X.fillna(med)

    # eliminar columnas constantes (VIF explota)
    nunique = X.nunique(dropna=True)
    keep = nunique[nunique > 1].index
    return X[keep]

# -------------------------
# Yeo-Johnson: transform + inversa
# -------------------------
def yj_transform(y: np.ndarray, lmbda: float) -> np.ndarray:
    y = np.asarray(y, dtype=float)
    return stats.yeojohnson(y, lmbda=lmbda)

def yj_fit_lambda(y_train: np.ndarray) -> float:
    """Estima lambda por MLE usando SOLO el array de train."""
    y_train = np.asarray(y_train, dtype=float)
    _, lmbda = stats.yeojohnson(y_train)  # devuelve (y_trans, lambda)
    return float(lmbda)

def yj_inverse_safe(y_t, lmbda, eps=1e-10):
    """
    Inversa Yeo-Johnson robusta:
    - respeta el rango real de YJ según lambda (evita bases <= 0)
    - evita overflow en exp() en los casos log (lambda≈0 o lambda≈2)
    """
    y_t = np.asarray(y_t, dtype=float)
    lam = float(lmbda)

    out = np.empty_like(y_t, dtype=float)
    pos = y_t >= 0

    # --- Rama y_t >= 0 ---
    if abs(lam) < 1e-12:
        # x = exp(y) - 1, pero cuidando overflow
        y_clip = np.clip(y_t[pos], a_min=None, a_max=np.log(np.finfo(float).max))
        out[pos] = np.expm1(y_clip)
    else:
        y_pos = y_t[pos].copy()

        # Si lambda < 0, el rango en y está acotado superiormente por -1/lambda
        if lam < 0:
            y_max = (-1.0 / lam) - eps
            y_pos = np.clip(y_pos, 0.0, y_max)

        base = lam * y_pos + 1.0
        base = np.maximum(base, eps)  # evita base <= 0 por redondeos
        out[pos] = np.power(base, 1.0 / lam) - 1.0

    # --- Rama y_t < 0 ---
    k = 2.0 - lam
    if abs(k) < 1e-12:
        # x = 1 - exp(-y), cuidando overflow en exp(-y)
        yneg = y_t[~pos]
        z = np.clip(-yneg, a_min=None, a_max=np.log(np.finfo(float).max))
        out[~pos] = 1.0 - np.exp(z)
    else:
        y_neg = y_t[~pos].copy()

        # Si lambda > 2 (k < 0), el rango en y está acotado inferiormente por 1/k
        if k < 0:
            y_min = (1.0 / k) + eps
            y_neg = np.clip(y_neg, y_min, 0.0)

        base = 1.0 - k * y_neg
        base = np.maximum(base, eps)
        out[~pos] = 1.0 - np.power(base, 1.0 / k)

    return out

# -------------------------
# VIF pruning (sin statsmodels)
# -------------------------
def _r2_of_regression(y: np.ndarray, X: np.ndarray) -> float:
    """R^2 de OLS via mínimos cuadrados con intercepto."""
    y = np.asarray(y, dtype=float)
    X = np.asarray(X, dtype=float)
    # intercepto
    X_ = np.column_stack([np.ones(len(X)), X])
    beta, *_ = np.linalg.lstsq(X_, y, rcond=None)
    y_hat = X_ @ beta
    ss_res = np.sum((y - y_hat) ** 2)
    ss_tot = np.sum((y - y.mean()) ** 2)
    return 0.0 if ss_tot == 0 else 1.0 - ss_res / ss_tot

def compute_vif_table(X: pd.DataFrame) -> pd.DataFrame:
    """Calcula VIF para cada columna de X."""
    X = X.copy()
    cols = list(X.columns)
    vifs = []
    X_values = X.values
    for i, col in enumerate(cols):
        y = X_values[:, i]
        X_others = np.delete(X_values, i, axis=1)
        if X_others.shape[1] == 0:
            vif = 1.0
        else:
            r2 = _r2_of_regression(y, X_others)
            vif = np.inf if (1.0 - r2) <= 1e-12 else 1.0 / (1.0 - r2)
        vifs.append(vif)
    return pd.DataFrame({"variable": cols, "vif": vifs}).sort_values("vif", ascending=False)

def vif_prune(
    X: pd.DataFrame,
    threshold: float = 10.0,
    min_features: int = 1,
    verbose: bool = True
):
    """Elimina iterativamente la variable con VIF más alto hasta cumplir threshold."""
    X_work = X.copy()
    dropped = []

    while True:
        if X_work.shape[1] <= min_features:
            break

        vif_tbl = compute_vif_table(X_work)
        max_vif = float(vif_tbl["vif"].iloc[0])
        worst = str(vif_tbl["variable"].iloc[0])

        if max_vif <= threshold:
            break

        dropped.append((worst, max_vif))
        X_work = X_work.drop(columns=[worst])

        if verbose:
            print(f"[VIF] drop: {worst} (VIF={max_vif:,.2f}) -> quedan {X_work.shape[1]} vars")

    return list(X_work.columns), dropped 

FUTURE_EXOG_METHOD = "last"  # "last" | "median" | "mode"

## Selección de exógenas por asset (solo con df_train, pre-CV)

In [6]:
# =========================
# Selección de exógenas por asset (solo con df_train, pre-CV)
# =========================
# Parámetros VIF
VIF_THRESHOLD = 5
VIF_MIN_FEATURES = 1
VIF_VERBOSE = True



col_map = {c.lower(): c for c in df_train.columns}

ASSETS_MANUAL = ["oro", "plata", "cobre", "gas_natural", "petroleo_brent"]
assets_from_csv = [a for a in sorted(EXOG_CANDIDATES_MAP.keys()) if a in col_map]

USE_ASSETS_FROM_CSV = True
ASSETS = assets_from_csv if USE_ASSETS_FROM_CSV else ASSETS_MANUAL

print("ASSETS:", ASSETS)

# -------------------------
# VIF pruning usando SOLO df_train
# -------------------------
selected_exogs = {}
vif_dropped_info = {}

for asset_key in ASSETS:
    # mapear a nombre real de columna (por si hay mayúsculas)
    asset_col = col_map.get(asset_key.lower(), asset_key)

    if asset_col not in df_train.columns:
        print(f"[WARN] Asset '{asset_col}' no está en df_train. Saltando.")
        continue

    # candidatos desde CSV
    cand = EXOG_CANDIDATES_MAP.get(asset_key.lower(), [])
    cand = [c for c in cand if c in df_train.columns]

    if len(cand) == 0:
        print(f"[WARN] Sin candidatos desde CSV para '{asset_key}'. Fallback: todas numéricas excepto target.")
        cand = [c for c in df_train.select_dtypes(include=[np.number]).columns if c != asset_col]

    X = _to_numeric_df(df_train[cand])

    if X.shape[1] == 0:
        print(f"[WARN] '{asset_key}': no quedaron exógenas numéricas tras limpieza.")
        selected_exogs[asset_key] = []
        vif_dropped_info[asset_key] = []
        continue

    keep_cols, dropped = vif_prune(X, threshold=VIF_THRESHOLD, min_features=VIF_MIN_FEATURES, verbose=VIF_VERBOSE)
    selected_exogs[asset_key] = keep_cols
    vif_dropped_info[asset_key] = dropped

    print(f"ASSET={asset_key} | candidatos={len(cand)} | keep={len(keep_cols)}")

selected_exogs

ASSETS: ['cobre', 'gas_natural', 'oro', 'petroleo_brent', 'plata']
[VIF] drop: dinero_circulante_china_m2_aprox_lcu (VIF=7,500.20) -> quedan 69 vars
[VIF] drop: dinero_circulante_mexico_m2_aprox_lcu (VIF=2,201.96) -> quedan 68 vars
[VIF] drop: dinero_circulante_india_m2_aprox_lcu (VIF=367.04) -> quedan 67 vars
[VIF] drop: pib_can (VIF=236.97) -> quedan 66 vars
[VIF] drop: dinero_circulante_japon_m2_aprox_lcu (VIF=169.71) -> quedan 65 vars
[VIF] drop: spy (VIF=9,284.07) -> quedan 64 vars
[VIF] drop: qqq (VIF=592.76) -> quedan 63 vars
[VIF] drop: dia (VIF=503.18) -> quedan 62 vars
[VIF] drop: pib_usa (VIF=308.86) -> quedan 61 vars
[VIF] drop: sp500 (VIF=303.63) -> quedan 60 vars
[VIF] drop: dinero_circulante_eeuu_m2_aprox_lcu (VIF=197.23) -> quedan 59 vars
[VIF] drop: msft (VIF=277.56) -> quedan 58 vars
[VIF] drop: brkb (VIF=137.18) -> quedan 57 vars
[VIF] drop: dax (VIF=115.38) -> quedan 56 vars
[VIF] drop: aapl (VIF=111.13) -> quedan 55 vars
[VIF] drop: produccion_gas_usa (VIF=102.07) 

{'cobre': ['gas_natural',
  'shanghai',
  'dxy_fut',
  'emergentes',
  'vix',
  'aem',
  'ag',
  'eqt',
  'fcx',
  'tasa_10y',
  'desempleo_can',
  'cercania_lunes',
  'lejanania_lunes_pas'],
 'gas_natural': ['shanghai',
  'dxy_fut',
  'vix',
  'amd',
  'aem',
  'ag',
  'eqt',
  'nem',
  'tasa_10y',
  'desempleo_can',
  'cercania_lunes',
  'lejanania_lunes_pas'],
 'oro': ['gas_natural',
  'shanghai',
  'dxy_fut',
  'emergentes',
  'vix',
  'aem',
  'ag',
  'eqt',
  'fcx',
  'tasa_10y',
  'desempleo_can',
  'cercania_lunes',
  'lejanania_lunes_pas'],
 'petroleo_brent': ['gas_natural',
  'shanghai',
  'dxy_fut',
  'vix',
  'amd',
  'aem',
  'ag',
  'eqt',
  'tasa_10y',
  'desempleo_can',
  'cercania_lunes',
  'lejanania_lunes_pas'],
 'plata': ['gas_natural',
  'shanghai',
  'dxy_fut',
  'vix',
  'amd',
  'aem',
  'ag',
  'eqt',
  'tasa_10y',
  'desempleo_can',
  'cercania_lunes',
  'lejanania_lunes_pas']}

In [7]:
selected_exogs

{'cobre': ['gas_natural',
  'shanghai',
  'dxy_fut',
  'emergentes',
  'vix',
  'aem',
  'ag',
  'eqt',
  'fcx',
  'tasa_10y',
  'desempleo_can',
  'cercania_lunes',
  'lejanania_lunes_pas'],
 'gas_natural': ['shanghai',
  'dxy_fut',
  'vix',
  'amd',
  'aem',
  'ag',
  'eqt',
  'nem',
  'tasa_10y',
  'desempleo_can',
  'cercania_lunes',
  'lejanania_lunes_pas'],
 'oro': ['gas_natural',
  'shanghai',
  'dxy_fut',
  'emergentes',
  'vix',
  'aem',
  'ag',
  'eqt',
  'fcx',
  'tasa_10y',
  'desempleo_can',
  'cercania_lunes',
  'lejanania_lunes_pas'],
 'petroleo_brent': ['gas_natural',
  'shanghai',
  'dxy_fut',
  'vix',
  'amd',
  'aem',
  'ag',
  'eqt',
  'tasa_10y',
  'desempleo_can',
  'cercania_lunes',
  'lejanania_lunes_pas'],
 'plata': ['gas_natural',
  'shanghai',
  'dxy_fut',
  'vix',
  'amd',
  'aem',
  'ag',
  'eqt',
  'tasa_10y',
  'desempleo_can',
  'cercania_lunes',
  'lejanania_lunes_pas']}

### Tabla final de exógenas seleccionadas (post-VIF)

variables se usaron para cada asset

In [8]:
rows = []
for asset_key, cols in selected_exogs.items():
    for v in cols:
        rows.append({"asset": asset_key, "variable": v})
df_selected_exogs = pd.DataFrame(rows).sort_values(["asset", "variable"]).reset_index(drop=True)

display(df_selected_exogs.head(30))
print("Total filas:", len(df_selected_exogs))

OUT_EXOG = (
    Path(config.PROCESSED_DATA_DIR)
    / "variables_seleccionadas_mensual"
    / "sarimax_var_diarias.csv"
)
df_selected_exogs.to_csv(OUT_EXOG, index=False)
print("saved:", OUT_EXOG)

,asset,variable
0,cobre,aem
1,cobre,ag
2,cobre,cercania_lunes
3,cobre,desempleo_can
4,cobre,dxy_fut
5,cobre,emergentes
6,cobre,eqt
7,cobre,fcx
8,cobre,gas_natural
9,cobre,lejanania_lunes_pas


Total filas: 62
saved: C:\Users\crsar\OneDrive - Universidad Católica de Chile\model\Proyect - Magister\code\afg_puc\data\processed\variables_seleccionadas_mensual\sarimax_var_diarias.csv


## Rolling-Origin CV (5 folds, horizonte 7) + Ljung-Box lag=10

- Cada fold: **train expanding** y **validación** de 7 días (paso 7).
- Métricas: MAPE y RMSE en **escala original** (usando inversa de YJ).
- Ljung-Box: se calcula con **residuos concatenados** de todos los puntos de validación (5×7 = 35 puntos), suficiente para `lag=10`.


In [9]:
def make_rolling_origin_splits(n_obs: int, horizon: int, n_folds: int):
    """Genera pares (train_idx, val_idx) con ventana expanding y step=horizon."""
    splits = []
    total_val = n_folds * horizon
    if n_obs <= total_val + 5:
        raise ValueError(f"No hay suficientes observaciones (n={n_obs}) para {n_folds} folds con horizon={horizon}.")
    for k in range(n_folds):
        train_end = n_obs - (n_folds - k) * horizon
        val_start = train_end
        val_end = train_end + horizon
        tr_idx = np.arange(0, train_end)
        va_idx = np.arange(val_start, val_end)
        splits.append((tr_idx, va_idx))
    return splits


# =========================
# Helpers: exógenas futuras sin "conocer" df_val / df_test
# =========================
FUTURE_EXOG_METHOD = "last"  # "last" | "median" | "mode"

def _prep_exog_train_and_future(X_tr: pd.DataFrame, horizon: int, method: str = FUTURE_EXOG_METHOD):
    """Limpia X_tr usando SOLO train y construye X_future (horizon x k) SIN usar X_val real."""
    if X_tr is None or X_tr.shape[1] == 0:
        return None, None, None

    # asegurar numérico + ffill + fillna mediana (solo con info del train del fold)
    X_tr = (
        X_tr.select_dtypes(include=[np.number])
            .replace([np.inf, -np.inf], np.nan)
            .ffill()
    )
    med = X_tr.median(numeric_only=True).fillna(0.0)
    X_tr = X_tr.fillna(med)

    # drop constantes para evitar singularidades / problemas numéricos
    nunique = X_tr.nunique(dropna=True)
    keep = nunique[nunique > 1].index
    X_tr = X_tr[keep]

    if X_tr.shape[1] == 0:
        return None, None, None

    meth = (method or "last").lower()

    if meth == "median":
        base_row = med.reindex(X_tr.columns).fillna(0.0)
    elif meth == "mode":
        mo = X_tr.mode(dropna=True)
        base_row = (mo.iloc[0] if len(mo) else med.reindex(X_tr.columns)).reindex(X_tr.columns).fillna(med.reindex(X_tr.columns).fillna(0.0))
    else:
        # "last": persistencia al último valor conocido del train del fold
        base_row = X_tr.iloc[-1].reindex(X_tr.columns).fillna(med.reindex(X_tr.columns).fillna(0.0))

    X_future = pd.DataFrame(
        np.repeat(np.asarray(base_row, dtype=float)[None, :], repeats=horizon, axis=0),
        columns=X_tr.columns,
    )
    return X_tr, X_future, base_row


def directional_ratio_from_base(y_true: np.ndarray, y_pred: np.ndarray, y_base: float) -> float:
    """Ratio Direccional usando base fija (última observación pre-pronóstico).
    Compara signo(y - base) entre real y pred. Base fija => NO usa y_{t-1} dentro del horizonte (no contaminación).
    """
    yt = np.asarray(y_true, dtype=float)
    yp = np.asarray(y_pred, dtype=float)
    b = float(y_base)

    d_true = np.sign(yt - b)
    d_pred = np.sign(yp - b)

    return float(np.mean(d_true == d_pred))


def rolling_origin_cv_sarimax(
    y: pd.Series,
    X: pd.DataFrame | None,
    order: tuple,
    seasonal_order: tuple,
    horizon: int = 7,
    n_folds: int = 5,
    yj_lambda_mode: str = "global",  
    enforce_stationarity: bool = False,
    enforce_invertibility: bool = False,
    maxiter: int = 50,
    lb_lag: int = 10,
    lb_alpha: float = 0.05,
    future_exog_method: str = FUTURE_EXOG_METHOD,
):
    """
    CV de SARIMAX:
    - Target en escala original (y) pero se ajusta en Yeo-Johnson
    - Métricas (MAPE/RMSE) en escala original (usando inversa)
    - Ratio Direccional (base fija = última obs pre-forecast por fold) => NO contaminado
    - AIC/BIC promedio (en escala transformada; es la que reporta statsmodels)
    - Ljung-Box lag=lb_lag sobre residuos CV concatenados (en escala original)
    - Exógenas futuras: NO se usan valores reales de validación; se construyen desde train del fold (last/median/mode)
    """
    y = y.dropna()
    if X is not None:
        X = X.loc[y.index]  # alinear

    splits = make_rolling_origin_splits(len(y), horizon=horizon, n_folds=n_folds)

    # lambda global (estimado SOLO con y de train completo pre-CV)
    if yj_lambda_mode == "global":
        lmbda_global = yj_fit_lambda(y.values)
    else:
        lmbda_global = None

    fold_metrics = []
    residuals_all = []
    aics, bics = [], []

    for fold, (tr_idx, va_idx) in enumerate(splits, start=1):
        y_tr = y.iloc[tr_idx]
        y_va = y.iloc[va_idx]

        # base fija (último dato pre-forecast) para ratio direccional
        y_base = float(y_tr.iloc[-1])
        base_ts = y_tr.index[-1] if hasattr(y_tr.index, "__len__") and len(y_tr.index) else None

        if X is not None:
            X_tr_raw = X.iloc[tr_idx]
        else:
            X_tr_raw = None

        # lambda por fold (sin leakage) o global
        if yj_lambda_mode == "fold":
            lmbda = yj_fit_lambda(y_tr.values)
        else:
            lmbda = lmbda_global

        # Transformación YJ (endog)
        y_tr_t = yj_transform(y_tr.values, lmbda)

        # Preparar exógenas: X_tr (limpio fold) + X_future (naive)
        X_tr, X_future, exog_base_row = (None, None, None)
        if X_tr_raw is not None:
            X_tr, X_future, exog_base_row = _prep_exog_train_and_future(X_tr_raw, horizon=len(y_va), method=future_exog_method)

        # Fit (en escala transformada)
        fit = SARIMAX(
            y_tr_t,
            exog=X_tr,
            order=order,
            seasonal_order=seasonal_order,
            enforce_stationarity=enforce_stationarity,
            enforce_invertibility=enforce_invertibility,
        ).fit(disp=False, maxiter=maxiter)

        if np.isfinite(getattr(fit, "aic", np.nan)):
            aics.append(float(fit.aic))
        if np.isfinite(getattr(fit, "bic", np.nan)):
            bics.append(float(fit.bic))

        # Forecast en escala transformada (SIN conocer exógenas futuras reales)
        if X_future is not None:
            fc_t = fit.forecast(steps=len(y_va), exog=X_future)
        else:
            fc_t = fit.forecast(steps=len(y_va))

        # Inversa a escala original
        fc = yj_inverse_safe(np.asarray(fc_t), lmbda)
        y_true = y_va.values

        rmse = float(np.sqrt(mean_squared_error(y_true, fc)))
        mape = float(mean_absolute_percentage_error(y_true, fc) * 100.0)

        # Ratio direccional (base fija)
        dir_ratio = directional_ratio_from_base(y_true, fc, y_base=y_base)

        fold_metrics.append({
            "fold": fold,
            "rmse": rmse,
            "mape": mape,
            "dir_ratio": dir_ratio,
            "y_base": y_base,
            "base_ts": base_ts,
            "future_exog_method": str(future_exog_method),
            "exog_base": (exog_base_row.to_dict() if exog_base_row is not None else None),
        })

        # residuos para Ljung-Box (en escala original)
        residuals_all.append(y_true - fc)

    residuals_all = np.concatenate(residuals_all)

    # Ljung-Box lag=lb_lag (H0: no autocorr). PASA si pvalue > alpha.
    lb_stat = np.nan
    lb_pvalue = np.nan
    lb_ok = np.nan
    if len(residuals_all) > lb_lag + 1:
        lb = acorr_ljungbox(residuals_all, lags=[lb_lag], return_df=True)
        lb_stat = float(lb["lb_stat"].iloc[0])
        lb_pvalue = float(lb["lb_pvalue"].iloc[0])
        lb_ok = bool(lb_pvalue > lb_alpha)

    df_folds = pd.DataFrame(fold_metrics)
    out = {
        "rmse_mean": float(df_folds["rmse"].mean()),
        "mape_mean": float(df_folds["mape"].mean()),
        "dir_ratio_mean": float(df_folds["dir_ratio"].mean()),
        "rmse_std": float(df_folds["rmse"].std(ddof=0)),
        "mape_std": float(df_folds["mape"].std(ddof=0)),
        "dir_ratio_std": float(df_folds["dir_ratio"].std(ddof=0)),
        "aic_mean": float(np.mean(aics)) if len(aics) else np.nan,
        "bic_mean": float(np.mean(bics)) if len(bics) else np.nan,
        "lb_stat_lag10": lb_stat,
        "lb_pvalue_lag10": lb_pvalue,
        "lb_ok_lag10": lb_ok,
        "folds": df_folds,
        "future_exog_method": str(future_exog_method),
    }
    return out

## Grid Search (SARIMAX) sobre CV

Define aquí tu grilla. Recomendación para diario:
- `s=5` semanal.

In [10]:
# =========================
# Grillas SARIMAX por commodity 
# =========================
# Importante:
# - Esto NO toca el split train/test.
# - Esto NO usa df_test para seleccionar variables ni para elegir la grilla.
# - Solo se usa df_train (con exógenas podadas por VIF) para CV y ranking.

HORIZON = 7
N_FOLDS = 5
YJ_LAMBDA_MODE = "global"   # lambda fijo estimado con todo df_train (pre-CV)

LB_LAG = 10
LB_ALPHA = 0.05
MAXITER = 50

# Grillas definidas originalmente en tu notebook (03a_model_diario.ipynb)
GRID_CFG = {
    "oro": {
        "p_vals": [0],
        "d_vals": [1],
        "q_vals": [0],
        "P_vals": [1, 2],
        "D_vals": [0],
        "Q_vals": [1, 2],
        "s_vals": [5],
    },
    "plata": {
        "p_vals": [1],
        "d_vals": [1],
        "q_vals": [1, 2],
        "P_vals": [1, 2],
        "D_vals": [0],
        "Q_vals": [0, 1],
        "s_vals": [5],
    },
    "cobre": {            #Probamos con ARIMA Simple producto de EDA 
        "p_vals": [2,3],  
        "d_vals": [1],
        "q_vals": [1,2],
        "P_vals": [0],
        "D_vals": [0],
        "Q_vals": [0],
        "s_vals": [0],
    }, 
    "gas_natural": {        #Probamos con ARIMA Simple producto de EDA 
        "p_vals": [0,1],
        "d_vals": [1],
        "q_vals": [0],
        "P_vals": [0],
        "D_vals": [0],
        "Q_vals": [0],
        "s_vals": [0],
    },
    "petroleo_brent": {
        "p_vals": [1],
        "d_vals": [1],
        "q_vals": [0, 1, 2],
        "P_vals": [0],
        "D_vals": [0],
        "Q_vals": [0, 1],
        "s_vals": [5],
    }}

def _normalize_int_list(vals, name="vals"):
    """Normaliza listas para SARIMAX:
    - int -> int
    - float entero -> int
    - float tipo 1.2 (typo común por olvidar coma) -> [1,2]
    """
    out = []
    for v in vals:
        if isinstance(v, (int, np.integer)):
            out.append(int(v))
            continue
        if isinstance(v, (float, np.floating)):
            # float casi entero
            if abs(v - round(v)) < 1e-9:
                out.append(int(round(v)))
                continue
            # typo 1.2 => [1,2]
            s = str(v)
            m = re.match(r"^(\d+)\.(\d)$", s)
            if m:
                out.extend([int(m.group(1)), int(m.group(2))])
                continue
            raise ValueError(f"{name}: valor no entero {v}. SARIMAX requiere enteros.")
        raise TypeError(f"{name}: tipo no soportado {type(v)} (valor={v}).")

    # mantener orden pero sin duplicados
    seen=set()
    out2=[]
    for x in out:
        if x not in seen:
            out2.append(x); seen.add(x)
    return out2

def build_grid(cfg: dict):
    p_vals = _normalize_int_list(cfg["p_vals"], "p_vals")
    d_vals = _normalize_int_list(cfg["d_vals"], "d_vals")
    q_vals = _normalize_int_list(cfg["q_vals"], "q_vals")
    P_vals = _normalize_int_list(cfg["P_vals"], "P_vals")
    D_vals = _normalize_int_list(cfg["D_vals"], "D_vals")
    Q_vals = _normalize_int_list(cfg["Q_vals"], "Q_vals")
    s_vals = _normalize_int_list(cfg["s_vals"], "s_vals")
    return list(product(p_vals, d_vals, q_vals, P_vals, D_vals, Q_vals, s_vals))

for k,v in GRID_CFG.items():
    g = build_grid(v)
    print(f"{k:14s} | combos={len(g)}")

oro            | combos=4
plata          | combos=8
cobre          | combos=4
gas_natural    | combos=2
petroleo_brent | combos=6


In [11]:
# =========================
# Ejecutar Grid Search por asset (CV 5 folds, horizon=7) usando exógenas podadas por VIF
# =========================
results_by_asset = {}

for asset_key in ASSETS:
    asset_col = col_map.get(asset_key.lower(), asset_key)

    if asset_col not in df_train.columns:
        print(f"[WARN] '{asset_col}' no está en df_train. Saltando.")
        continue

    cfg = GRID_CFG.get(asset_key.lower())
    if cfg is None:
        print(f"[WARN] Sin grilla definida para '{asset_key}'. Saltando.")
        continue

    grid = build_grid(cfg)

    # y y exógenas SOLO desde df_train (NO se usa df_test aquí)
    y = df_train[asset_col]

    exogs = selected_exogs.get(asset_key, [])
    X = None
    if len(exogs) > 0:
        X = _to_numeric_df(df_train[exogs])

    rows = []
    for (p, d, q, P, D, Q, s) in tqdm(grid, desc=f"grid {asset_key}", leave=False):
        res = rolling_origin_cv_sarimax(
            y=y,
            X=X,
            order=(p, d, q),
            seasonal_order=(P, D, Q, s),
            horizon=HORIZON,
            n_folds=N_FOLDS,
            yj_lambda_mode=YJ_LAMBDA_MODE,
            maxiter=MAXITER,
            lb_lag=LB_LAG,
            lb_alpha=LB_ALPHA,
            future_exog_method=FUTURE_EXOG_METHOD,
        )
        rows.append({
            "asset": asset_key,
            "order": (p, d, q),
            "seasonal_order": (P, D, Q, s),
            "mape_mean": res["mape_mean"],
            "rmse_mean": res["rmse_mean"],
            "mape_std": res["mape_std"],
            "rmse_std": res["rmse_std"],
            "dir_ratio_mean": res.get("dir_ratio_mean", np.nan),
            "dir_ratio_std": res.get("dir_ratio_std", np.nan),
            "aic_mean": res["aic_mean"],
            "bic_mean": res["bic_mean"],
            "lb_pvalue_lag10": res["lb_pvalue_lag10"],
            "lb_ok_lag10": res["lb_ok_lag10"],
        })

    df_grid = pd.DataFrame(rows).sort_values(by=["dir_ratio_mean", "mape_mean"], ascending=False).reset_index(drop=True)
    results_by_asset[asset_key] = df_grid

    print(f"\n===== {asset_key.upper()} | combos={len(df_grid)} | exogs={len(selected_exogs.get(asset_key, []))} =====")
    display(df_grid)


===== COBRE | combos=4 | exogs=13 =====


,asset,order,seasonal_order,mape_mean,rmse_mean,mape_std,rmse_std,dir_ratio_mean,dir_ratio_std,aic_mean,bic_mean,lb_pvalue_lag10,lb_ok_lag10
0,cobre,"(3, 1, 2)","(0, 0, 0, 0)",4.574173,0.301123,3.565733,0.255194,0.771429,0.264961,"-18,814.134782","-18,691.585120",0.065298,True
1,cobre,"(2, 1, 2)","(0, 0, 0, 0)",4.572402,0.301061,3.560601,0.254924,0.771429,0.264961,"-18,816.141995","-18,700.042315",0.065698,True
2,cobre,"(3, 1, 1)","(0, 0, 0, 0)",4.565815,0.300572,3.569407,0.255442,0.771429,0.264961,"-18,816.423636","-18,700.323957",0.065719,True
3,cobre,"(2, 1, 1)","(0, 0, 0, 0)",92.778230,7.623937,36.473923,3.383425,0.228571,0.264961,"-11,097.349947","-10,987.696614",0.104878,True



===== GAS_NATURAL | combos=2 | exogs=12 =====


,asset,order,seasonal_order,mape_mean,rmse_mean,mape_std,rmse_std,dir_ratio_mean,dir_ratio_std,aic_mean,bic_mean,lb_pvalue_lag10,lb_ok_lag10
0,gas_natural,"(1, 1, 0)","(0, 0, 0, 0)",3.643990,0.149442,1.799423,0.075685,0.371429,0.232115,"-36,361.468143","-36,271.162404",0.031928,False
1,gas_natural,"(0, 1, 0)","(0, 0, 0, 0)",3.578342,0.147138,1.764068,0.075373,0.314286,0.277010,"-36,326.542440","-36,242.687111",0.042943,False



===== ORO | combos=4 | exogs=13 =====


,asset,order,seasonal_order,mape_mean,rmse_mean,mape_std,rmse_std,dir_ratio_mean,dir_ratio_std,aic_mean,bic_mean,lb_pvalue_lag10,lb_ok_lag10
0,oro,"(0, 1, 0)","(1, 0, 1, 5)",1.161124,44.148529,0.622790,21.268498,0.685714,0.189521,"-48,126.077535","-48,022.888091",0.001604,False
1,oro,"(0, 1, 0)","(2, 0, 2, 5)",1.161481,44.154495,0.622911,21.286396,0.657143,0.294161,"-48,080.430351","-47,964.361501",0.001581,False
2,oro,"(0, 1, 0)","(1, 0, 2, 5)",1.161459,44.154124,0.623048,21.290610,0.628571,0.279942,"-48,082.826026","-47,973.205445",0.001582,False
3,oro,"(0, 1, 0)","(2, 0, 1, 5)",1.160490,44.122754,0.621963,21.246615,0.628571,0.279942,"-48,094.209980","-47,984.585757",0.001620,False



===== PETROLEO_BRENT | combos=6 | exogs=12 =====


,asset,order,seasonal_order,mape_mean,rmse_mean,mape_std,rmse_std,dir_ratio_mean,dir_ratio_std,aic_mean,bic_mean,lb_pvalue_lag10,lb_ok_lag10
0,petroleo_brent,"(1, 1, 2)","(0, 0, 1, 5)",2.073514,1.755490,1.389141,1.201065,0.514286,0.232115,"1,825.334422","1,934.965928",0.280019,True
1,petroleo_brent,"(1, 1, 2)","(0, 0, 0, 5)",2.085462,1.787239,1.364422,1.179082,0.485714,0.264961,"1,838.283772","1,941.483487",0.280862,True
2,petroleo_brent,"(1, 1, 0)","(0, 0, 0, 5)",2.073527,1.779605,1.365766,1.179559,0.485714,0.264961,"1,834.559604","1,924.865343",0.284694,True
3,petroleo_brent,"(1, 1, 0)","(0, 0, 1, 5)",2.069086,1.751710,1.389560,1.201709,0.485714,0.232115,"1,822.088974","1,918.829078",0.281035,True
4,petroleo_brent,"(1, 1, 1)","(0, 0, 0, 5)",2.049411,1.742855,1.384940,1.185379,0.485714,0.264961,"1,834.920242","1,931.673183",0.267374,True
5,petroleo_brent,"(1, 1, 1)","(0, 0, 1, 5)",2.054888,1.732624,1.393259,1.200617,0.457143,0.228571,"1,825.433947","1,928.619967",0.282226,True



===== PLATA | combos=8 | exogs=12 =====


,asset,order,seasonal_order,mape_mean,rmse_mean,mape_std,rmse_std,dir_ratio_mean,dir_ratio_std,aic_mean,bic_mean,lb_pvalue_lag10,lb_ok_lag10
0,plata,"(1, 1, 2)","(1, 0, 0, 5)",1.978384,0.883869,1.035690,0.438280,0.742857,0.209956,"-38,658.121068","-38,548.482283",0.027879,False
1,plata,"(1, 1, 2)","(1, 0, 1, 5)",1.970449,0.882768,1.032073,0.436843,0.742857,0.209956,"-38,644.552111","-38,528.471693",0.028333,False
2,plata,"(1, 1, 1)","(1, 0, 1, 5)",1.989579,0.892474,1.065121,0.448538,0.657143,0.171429,"-38,653.894858","-38,544.259713",0.026362,False
3,plata,"(1, 1, 1)","(1, 0, 0, 5)",1.997589,0.893487,1.068867,0.450079,0.628571,0.193781,"-38,664.864912","-38,561.675467",0.026132,False
4,plata,"(1, 1, 2)","(2, 0, 1, 5)",2.008048,0.895035,1.045338,0.442887,0.600000,0.139971,"-38,621.982123","-38,499.465002",0.025913,False
5,plata,"(1, 1, 2)","(2, 0, 0, 5)",2.007789,0.893263,1.041881,0.441295,0.542857,0.318158,"-38,617.522017","-38,501.453166",0.026031,False
6,plata,"(1, 1, 1)","(2, 0, 1, 5)",2.026808,0.904697,1.078776,0.454635,0.542857,0.057143,"-38,626.580944","-38,510.512093",0.024211,False
7,plata,"(1, 1, 1)","(2, 0, 0, 5)",2.026681,0.902865,1.075342,0.453097,0.514286,0.069985,"-38,622.128625","-38,512.508043",0.024488,False


## Interpretación rápida de Ljung-Box (lag=10)

- La hipótesis nula **H0** es “no hay autocorrelación” en los residuos.  
- Si `pvalue > 0.05` (típico), **no** rechazas H0 → *residuos más “blancos”* (mejor).
- Si `pvalue <= 0.05`, hay evidencia de autocorrelación → el modelo puede estar dejando estructura sin capturar.
